RIVERDALE GENERAL HOSPITAL

This is a simple model, built in Python, of the emergency room discussed in the WICI workshop.

We're looking to see how we can optimally use the given resources to optimise the wait times in the ER, specifically the time between triage and doctor consultation for all patients, but especially the most serious patients (judjed by triage).

There are many tools for visualising ABMs, such as those found on the sites http://www.agent-based-models.com/blog/resources/simulators/ and https://en.wikipedia.org/wiki/Comparison_of_agent-based_modeling_software. Just google "agent based model gui"; that should do it. I'm not familiar with any of those tools: I prefer looking at a sea of code, but hopefully I'll learn one of them for fun.

README !!!!!! 

To run the code, especially after changing parameter values, go to the "Kernel" option in the toolbar above, and hit "Restart and run all". The reason we have to burn everything and start over is that the notebook stores variables and values for later use. So, you could end up using old values instead of new ones, especially if you evaluate it box-by-box. Start fresh every time.

#-----------------------------------------------------------------------------------------------------

This first cell just imports libraries for functions and data structures we will use.

In [1]:
# Brendon Phillips
# PhD candidate
# Department of Applied Mathematics
# Faculty of Mathematics
# University of Waterloo

# Agent-based model of an Emergency Room
# importing the libraries with the functions we need

# a collection of data structures
import collections

# a set of mathematical functions used
import math

# defines the 'queue' data structure, used to preserve priority (first in, first out)
import queue

# this implements a "priority queue". it's a normal queue (first in, first out), but also accounts for priority
# so, objects with higher priority always come out first, and then lower-priority objects after them
# we will use this for the queues: waiting for a bed, and waiting to see the doctor
# the patients with lower ranks in triage will get priority, ahead of the less serious patients
import heapq

# this library gives us different distribution of random numbers to be used in the simulation
import numpy.random

The next cell gives the parameter values of the simulation. The names should be fairly self-explanatory. Feel free to change these values for the simulation, and see how the output of the simulation changes.

In [2]:
# each time step represents 5 minutes. this relates to the time step chosen during the workshop
# always choose the time step as the shortest process occurring in each time step
# in this case, the shortest process is by the receptionists, who we assume take 5 minutes
# therefore each time step in the model represents 5 minutes

# this represents 6 months, and takes a looong time to run. feel free to reduce it
number_of_time_steps_in_the_simulation = 52560


# this is the time at which each an agent leaves the ER flow due to frustration at the length of the wait
# feel free to change this value
# remember that each time step represents 5 minutes

frustration_threshold_measured_in_time_steps = 96 # represents 8 hours (96 5-minute chunks)

# the number of time steps each nurse takes to triage each patient
max_nurse_triage_time = 2

number_of_beds_in_the_ER = 20

# this refers to the number of doctors in the ER.
number_of_doctors = 8

number_of_receptionists = 3

number_of_triage_nurses  = 6

Find below the lists that we will use to store various queues in the program, to keep the patients in order of arrival or priority. 

A Deque (pronounced 'deck', short for double-ended queue) is a structure that keeps the order of the objects that it holds, as well as allowing removal from both the left and right sides (rather than a queue, which only allows removal from the front - think of the lines in the supermarket).

The reason I used a deque (rather than a queue or list) is so that we can easily remove any patients that become frustrated or die. With a simple queue, we'd have to remove all the elements in front of it, but the deque's'Python implementation is easier to use (other programming languages are different).

Priority queues are queues (first-in, first-out), but they allow objects of greater "importance" (priority) to skip to the front of the queue, so that the highest-priority objects are always removed first, before others. This is important for assigning beds and doctors, since the more serious cases in the ER will be seen first.

In [3]:
# a deque (double ended queue) allows us to represent a queue (first in, first out), as well as removing any element we want from the queue
# we'll remove any agent that gets frustrated at the wait, and anyone that dies during the simulation
waiting_for_Registration = collections.deque()

# the same as above
waiting_for_Triage = collections.deque()

# we'll use this as a "priority queue"; first-in, first-out, but objects with a higher priority always come out first, ahead of lower-priority objects
triaged_and_waiting_for_a_bed = []

# the same as the above. more serious cases will be given beds in the ER bay faster than lower prioritry patients
in_Bed_waiting_for_MD = []

# these lines just change the lists above into the priority queues that we want.
# don't worry too much about this. there's a looooong comp-sciency explanation here
heapq.heapify(triaged_and_waiting_for_a_bed)
heapq.heapify(in_Bed_waiting_for_MD)


The cell object below is the "Patient" agent. Remember that agents (in general) must be:

1) autonomous - capable of independent action
2) self-contained - the patient has all its information. no one else owns it
3) capable of passive or active response

A "class" is a simple template for what each agent is. You say what properties the agent has and what it does, as an idea. The actual object is created later.

Many of these qualities may not seem important now, but will be important to gathering data.

A sigmoid curve (referrenced below) is a curve similar to \[f(x)=\frac1{1+e^{-x}}\]

In [4]:
class Patient:

    def __init__(self):

        # the patient pulls a number when they enter the waiting room
        self.patient_number = -1

        # tells the time that the agent arrived to the ER
        self.time_of_arrival = -1

        # each patient will be assigned a rank by the nurse during triage
        # triage ranks aare between 1 and 4 inclusive, with 1 being the most serious, and 4 being the least serious
        # the initial -1 value is a "placeholder"; a nonsense value. it if pops up, we know we've done something wrong, and the code will crash
        self.triage_rank = -1

        # this marks the time that the patient is finally registered, and the same for the other numbers
        self.done_reception = 0

        # time that patient gets through the triage, and is assigned a rank
        self.done_triage = 0

        # the time that the patient get a bed in the ER bay
        self.time_got_a_bed = 0

        # the time that the patient sees the doctor
        self.saw_the_doctor = 0

        # the time that the patient exited the simulation, either by death, frustration, or discharge (after seeing the doctor)
        self.time_exited = 0

        # how the patient arrives to the ER
        # "A" for ambulance, and "W" for walk-in
        self.mode_of_arrival = '\0'

        # tracks the total time spent waiting by the patient
        self.total_wait_time = 0

    # this function defines what is means for two objects to be "equal", hence the name "eq"
    # it's a good idea to include this for objects in every class
    # this function says that two patients ("self" and "other") are equal if *all* their proporties are equal
    def __eq__(self, other):
        return ((self.patient_number	== other.patient_number) and \
            (self.triage_rank 			== other.triage_rank) and \
            (self.time_of_arrival		== other.time_of_arrival) and \
            (self.done_reception		== other.done_reception) and \
            (self.fone_triage			== other.done_triage) and \
            (self.time_got_a_bed		== other.time_got_a_bed) and \
            (self.saw_the_doctor		== other.saw_the_doctor) and \
            (self.time_exited			== other.time_exited) and \
            (self.mode_of_arrival		== other.mode_of_arrival) and \
            (self.total_wait_time		== other.wait_time))

    # this function defines what it means for one object ("self") to be "less than" another one ("other")
    # we discussed the "priority queue" before. the most serious patients are pulled from the queue before the less serious ones
    # therefore, to sort the priority queue, we'll say that one object is "less than" the other if it has a lower triage number
    # reminder: the lower numbers are the most serious ones, as is with numbered triage scales
    # "T1" is more serious than "T2", which is more serious than "T3", and so on...
    # objects with lower priority numbers exit the queue first, so this is exactly what we want
    def __lt__(self, other):
        return (self.triage_rank < other.triage_rank)

    # gives the number assigned to the patient when they arrived to the ER
    # this is NOT a priority score; it's just a regular number for ordering patients
    def patient_number(self):
        return self.patient_number

    # function telling whether the agent should get frustrated and exit the simulation, or stay in the ER flow for another time step
    # this is defined by the threshold set in the model parameter cell above
    def should_I_leave(self):
        return (self.total_wait_time > frustration_threshold_measured_in_time_steps)

    # calculates and return the probability that an agent will die during the simulation
    # in line with some research papers (references given at the end), the probability of death depends on:
    #    . the severity of the complaint (triage rank)
    #    . the length of time waited before receiving adequate medical attention (total wait time)
    # for now, we'll assume that patients don't die during treatment. it's a bad assumption, but it's in line with other research models
    # always keep in mind that trying to model the real world *exactly* will give you a useless model, with no power of prediction
    def death_probability(self):

        # assume that the patient can't die before triage; it's a simplifying assumption
        if self.triage_rank == -1: return 0.0

        # this is a nonsense function
        # it's based on the shape of a sigmoid curve, and scaled so that lower priority results in lower probability,
        # and higher priority or longer wait times result in higher probabilities of death
        # usually, instead of a plain function, we'd gather data of the death rates of patients in each triage category. and use that for the model
        # for example:
        #
        # if self.triage_rank == 1:
        #     return rate specific to the most serious patients
        # elif self.triage_rank = 2:
        #     etc...

        return 0.00001/(1 + numpy.exp(-self.total_wait_time))*(3/715*self.total_wait_time + 87/286)*(-33*self.triage_rank/100 + 133/100)

This is the description of the "Triage Nurse" agent in the simulation. This agent keeps track of the patient it's seeing, and the time spent on the patient. When the consultation is done (the time spent hits the threshold parameter value given above), the consultation ends.

There is no "ER Nurse" agent, since they work with the doctors, so that the doctor-nurse pair in the ER can be represented as a single object. It's called "Doctor", further below.

In [5]:
class Triage_Nurse:

    # this function is used to create the agent
    def __init__(self):
        # time spent on the current patient
        self.__triage_time = 0

        #the patient currently being seen
        self.__is_currently_seeing_patient = Patient()

    # tells whether the nurse is triaging a patient at the moment, or not
    def is_busy(self):
        return (self.__triage_time != 0)

    # tells the agent to start triaging the given patient
    # this starts the triage_time counter, and assigns the patient being seen
    def triage_patient(self, person):
        self.__triage_time = 1
        self.__is_currently_seeing_patient = person

    # if the consultation has not run for the full length, this function tels the object to add 1 to the consultation time, without swapping out the patient
    def continue_triage(self):
        self.__triage_time += 1

    # tells the agent to finish with the current patient, release them, and reset the time counter
    def finish_triage(self):
        self.__is_currently_seeing_patient = Patient() # Patient() is a placeholder value
        self.__triage_time = 0

    # return the time spent on the current patient
    def time_spent(self):
        return self.__triage_time

    # tell which patient the nurse is currently seeing to
    def currently_seeing(self):
        return self.__is_currently_seeing_patient


This is the "Doctor" class. The "Doctor" and "Triage Nurse" agents in this simulation function almost the same way.

In [6]:
class Doctor:

    def __init__(self):
        self.treatment_time = 0
        self.is_currently_seeing_patient = Patient()

    def is_busy(self):
        return (self.treatment_time != 0)

    def treat_patient(self, person):
        self.treatment_time = 1
        self.is_currently_seeing_patient = person

    def continue_treatment(self):
        self.treatment_time += 1

    def finish_treatment(self):
        self.is_currently_seeing_patient = Patient()
        self.treatment_time = 0

    def time_spent(self):
        return self.treatment_time

    def currently_treating(self):
        return self.is_currently_seeing_patient

    # returns the triage rank of the patient currently being seen
    # this function is used to calculate the treatment time given to each patient
    def triage_rank_of_patient(self):
        return self.is_currently_seeing_patient.triage_rank


These three lists catch the Patient agents leaving the flow of the ER, either from death, frustration, or discharge (after the doctor is finished).

In [7]:
Dead = []
Discharged = []
Left_Without_Being_Seen = []

This function adds 1 to the wait times of all agents, and removes the agents that are frustrated (total wait time has hit the frustration threshold parameter). Patients are only allowed to leave the simulation this way, if they have triage numbers 3 or 4. 

We're assuming that those patients with numbers 1 and 2 will not leave the ER, since they have higher priority in all the queues, and are serious enough to require medical care.

In [8]:
def increment_waiting_time_and_remove_frustrated_patients(D, time_step):

    global Left_Without_Being_Seen

    # we want to find the position in the list where frustrated patients are, that is:
    # 1. their wait time is above the frustration threshold
    # 2. the triage rank is greater than 2
    index = 0

    while index != -1:

        # find the position of the agent. if there aren't any more frustrated patients, skip to the end of the function
        try:
            index = D.index(next( z for z in D if (z.should_I_leave() and z.triage_rank > 2)  ))
        except StopIteration:
            index = -1
            break

        # remove the frustrated patient
        Frustrated_Patient = D[index]
        D.remove(Frustrated_Patient)
        Frustrated_Patient.time_exited = time_step
        Left_Without_Being_Seen.append( Frustrated_Patient )

    # for every other non-frustrated patient, increment their wait time by 1
    for patient in D:
        patient.total_wait_time += 1

    # make sure that we are still accounting for the priorities of the patients
    if type(D) == list:
            heapq.heapify(D)

This function removes all the dead patient from the lists given (D). Remember that the probability of death is based on both the triage rank of the patient, and the total time spend waiting in the ER flow.

In [9]:
def remove_dead_patients(D, time_step):

    global Dead
    index = 0
    while index != -1:

        # find the positions of the nodes marked for death
        try:
            index = D.index(next( z for z in D if (numpy.random.uniform() < z.death_probability())  ))
        except StopIteration:
            index = -1
            break

        # legit kill the patient and add them to the death list
        Dead_Patient = D[index]
        D.remove(Dead_Patient)
        Dead_Patient.time_exited = time_step
        Dead.append( Dead_Patient )


Tis just creates a list of all the "Triage Nurse" and Doctor agents used in the simulation. We can do this beforehand, since the number of doctors and nurses does not change during the simulation.

For the Patients, we'll create those on the fly.

In [10]:
Pool_of_Triage_Nurses	= [ Triage_Nurse() for i in range(number_of_triage_nurses) ]
Pool_of_Doctors         = [ Doctor() for i in range(number_of_doctors) ]

The following function tells the doctor the time length of the treatment for each patient. Notice that treatment times are larger for more serious cases, wile those less serious are shorter.

In [11]:
# remember that each time step represents 5 minutes
def treatment_time_for_this_case(triage_rank):
    if triage_rank == 1: return 60 # 5 hours
    if triage_rank == 2: return 30 # 2.5 hours 
    if triage_rank == 3: return 12 # 1 hour
    if triage_rank == 4: return 6 # half an hour
    else: return None

This tracks the number of patients currently in the simulation, and gives each Patient agent a new number when it enters.

In [12]:
number_of_total_patients = 0

def get_new_patient_number():
    global number_of_total_patients

    # increment the nunber of patients in the model, and return the next patient number
    number_of_total_patients += 1;
    return number_of_total_patients


Finally, we get to the actual simulation. Since I can't break it up into separate cells, I'll write hella comments.

Quick rundown - in each timestep we:
1)  process walk-in and ambulance patients. ambulance arrivals skip registration and go straight to triage.
2)  patients pass through registration, and are directed to the triage queue
3)  each nurse will take a new patient once they are not busy, and patients are available
4)  after triage, patients will wait for a bed, with the more serious patients given higher priority
5)  each doctor will take a new patient, once they are available, and patients are still waiting
6)  frustrated and dead patients will be removed

In [13]:
# for all time steps in the simulation
for tau in range(number_of_time_steps_in_the_simulation):

    # We use a Poisson process to model patient arrivals.
    # modes of arrival: "W" - walk-in, and 'A' - ambulance

    # all the walk-in patients are given a new number enqueued to see the receptionists
    for a_walk_in_patient in range( numpy.random.poisson(1) ):

        New_Patient = Patient() # create an new agent
        New_Patient.patient_number = get_new_patient_number()
        New_Patient.time_of_arrival = tau
        New_Patient.mode_of_arrival = 'W'
        waiting_for_Registration.append( New_Patient ) # add it to the registration queue



    # all the ambulance arrivals are sent immediately to triage
    # assume that the ambulances bring patients only at the top of every hour
    # tau will be a multiple of 12 at the start of every hour (each time step is 5 mins, and 60 mins per hour) - 12 time steps per hour
    if tau%12 == 0:
        for person in range( numpy.random.poisson(1) ):

            Ambulance_Patient = Patient() # create a new agent
            Ambulance_Patient.patient_number = get_new_patient_number()
            Ambulance_Patient.time_of_arrival = tau
            Ambulance_Patient.mode_of_arrival = 'A'
            waiting_for_Triage.append( Ambulance_Patient ) # send them straight to triage



    # each receptionist will process a single patient each time step
    for i in range(number_of_receptionists):

        # if there is no-one else to be registered, move on
        if not waiting_for_Registration :
            break

        # register the patient, and send them for triage
        Just_Got_Registered = waiting_for_Registration.popleft()
        Just_Got_Registered.done_reception = tau
        waiting_for_Triage.append(Just_Got_Registered)



    # for each available nurse
    for RN in Pool_of_Triage_Nurses:

        # break the loop is there is no-one waiting for triage
        if not waiting_for_Triage :
            break

        # if this nurse is busy
        if RN.is_busy():

            # if triage time is not done, continue (add 1 to the consultation time)
            if RN.time_spent() < max_nurse_triage_time :
                RN.continue_triage()

            # otherwise (if the triage is done)
            else:
                # assign a triage rank to the patient
                Old_Patient = RN.currently_seeing()

                # we assume that serious complaints are less common than people overreacting to a bad cold
                # so, we want a triage distribution heavily skewed towards giving lower triage scores
                assigned_rank = int(numpy.round(numpy.random.poisson(4)))

                # all triage nunbers must be less than or equal to 4
                assigned_rank = 4 if (assigned_rank >= 4) else assigned_rank

                # similarly, all nunbers must be more than 0
                assigned_rank = 1 if assigned_rank == 0 else assigned_rank

                # finally, give the number
                Old_Patient.triage_rank = assigned_rank

                # send old patient to go wait for an available bed in the ER bay
                heapq.heappush(triaged_and_waiting_for_a_bed, Old_Patient)
                Old_Patient.done_triage = tau

                # tell the nurse to finish with this patient, and start triaging a new one immediately
                # simplifying assumption - nurses and doctors do not take breaks
                RN.finish_triage()
                New_Patient = waiting_for_Triage.popleft()
                RN.triage_patient( New_Patient )

        # otherwise, if the nurse is not busy
        else:
            # get a new patient and start triaging
            New_Patient = waiting_for_Triage.popleft()
            RN.triage_patient( New_Patient )



    # while there are still empty beds available in the ER bay
    # i.e., the number of available beds is less than the max number given in the parameters above
    while len(in_Bed_waiting_for_MD) < number_of_beds_in_the_ER:

        # break the loop if there is no-one waiting for a bed
        if not triaged_and_waiting_for_a_bed:
            break
        # get the patient in the list with the highest priority, and put them in a bed
        Patient_Getting_a_Bed = heapq.heappop(triaged_and_waiting_for_a_bed)
        heapq.heappush(in_Bed_waiting_for_MD, Patient_Getting_a_Bed)
        Patient_Getting_a_Bed.time_got_a_bed = tau



    # for each doctor in the pool
    for MD in Pool_of_Doctors:

        # break the loop is there is no-one waiting for a doctor
        if not in_Bed_waiting_for_MD:
            break

        # if the doctor is currently with someone
        if MD.is_busy():

            # if the current treatment is not complete, continue with no change
            if MD.time_spent() < treatment_time_for_this_case( MD.triage_rank_of_patient() ):
                MD.continue_treatment()

            # if treatment is complete
            else:

                # discharge the current patient
                Done_Patient = MD.currently_treating()
                Discharged.append( Done_Patient )
                Done_Patient.time_exited = tau
                MD.finish_treatment()

                # immediately take the next patient (in bed) with the highest priority
                New_Patient = heapq.heappop(in_Bed_waiting_for_MD)
                MD.treat_patient( New_Patient )
                New_Patient.saw_the_doctor = tau;

        # if the doctor is not currently seeing a patient
        else:

            # immediately choose a new patient (in bed) with the highest priority
            New_Patient = heapq.heappop(in_Bed_waiting_for_MD)
            MD.treat_patient( New_Patient )
            New_Patient.saw_the_doctor = tau

    # go through all the waiting lists, and fire the frustrated patients
    increment_waiting_time_and_remove_frustrated_patients(waiting_for_Triage, tau)
    increment_waiting_time_and_remove_frustrated_patients(waiting_for_Registration, tau)
    increment_waiting_time_and_remove_frustrated_patients(triaged_and_waiting_for_a_bed, tau)

    # move all dead patients to the morgue
    remove_dead_patients(waiting_for_Registration, tau)
    remove_dead_patients(waiting_for_Triage, tau)
    remove_dead_patients(triaged_and_waiting_for_a_bed, tau)
    remove_dead_patients(in_Bed_waiting_for_MD, tau)


In [14]:
# print the stats of the run

print("\ttotal number of patients: {}".format(number_of_total_patients))
print("\twaiting for registration: {}".format( len(waiting_for_Registration) ))
print("\twaiting for triage: {}".format( len(waiting_for_Triage) ))
print("\tbeing triaged: {}".format(len(list( filter(lambda x: x.is_busy(), Pool_of_Triage_Nurses) ))) )
print("\ttriaged and waiting for a bed: {}".format(len(triaged_and_waiting_for_a_bed)) )
print("\tin bed waiting for a doctor: {}".format( len(in_Bed_waiting_for_MD) ))
print("\tnumber being seen by doctor: {}".format( len(list( filter(lambda x: x.is_busy(), Pool_of_Doctors) ))) )
print("\tnumber discharged: {}".format( len(Discharged) ) )
print("\tnumber left due to frustration: {}".format( len(Left_Without_Being_Seen) ))
print("\tnumber dead: {}".format( len(Dead) ))


	total number of patients: 5377
	waiting for registration: 0
	waiting for triage: 0
	being triaged: 6
	triaged and waiting for a bed: 475
	in bed waiting for a doctor: 20
	number being seen by doctor: 8
	number discharged: 839
	number left due to frustration: 3998
	number dead: 31


In [15]:
# average wait time for discharged patients

# each time step represents 5 minutes

cumulative_wait_time = 0

for patient in Discharged:
    cumulative_wait_time += patient.total_wait_time*5

print("average wait time for discharged patients in minutes: {}".format( cumulative_wait_time/len(Discharged) ))

average wait time for discharged patients: 309.25268176400476


In [16]:
# average wait times from triage to treatment, based on rank

num_rank_1 = 0
num_rank_2 = 0
num_rank_3 = 0
num_rank_4 = 0

cumulative_time_from_triage_to_MD_rank_1 = 0
cumulative_time_from_triage_to_MD_rank_2 = 0
cumulative_time_from_triage_to_MD_rank_3 = 0
cumulative_time_from_triage_to_MD_rank_4 = 0

for patient in Discharged:

    wait_time = (patient.saw_the_doctor - patient.done_triage)

    # each time step represents 5 minutes

    if patient.triage_rank == 1:
        num_rank_1 += 1
        cumulative_time_from_triage_to_MD_rank_1 += wait_time*5

    elif patient.triage_rank == 2:
        num_rank_2 += 1
        cumulative_time_from_triage_to_MD_rank_2 += wait_time*5

    elif patient.triage_rank == 3:
        num_rank_3 += 1
        cumulative_time_from_triage_to_MD_rank_3 += wait_time*5

    elif patient.triage_rank == 4:
        num_rank_4 += 1
        cumulative_time_from_triage_to_MD_rank_4 += wait_time*5

print("average times from triage to MD for DISCHARGED patients:\n")
if num_rank_1 != 0: print("\trank 1 patients: number: {}, time in minutes: {}".format( num_rank_1, cumulative_time_from_triage_to_MD_rank_1 / num_rank_1 ))
if num_rank_2 != 0: print("\trank 2 patients: number: {}, time in minutes: {}".format( num_rank_2, cumulative_time_from_triage_to_MD_rank_2 / num_rank_2 ))
if num_rank_3 != 0: print("\trank 3 patients: number: {}, time in minutes: {}".format( num_rank_3, cumulative_time_from_triage_to_MD_rank_3 / num_rank_3 ))
if num_rank_4 != 0: print("\trank 4 patients: number: {}, time in minutes: {}".format( num_rank_4, cumulative_time_from_triage_to_MD_rank_4 / num_rank_4 ))

average times from triage to MD for DISCHARGED patients:

	rank 1 patients: number: 498, time in minutes: 84.40763052208835
	rank 2 patients: number: 317, time in minutes: 4021.9085173501576
	rank 3 patients: number: 16, time in minutes: 30.0
	rank 4 patients: number: 8, time in minutes: 13.125


There *may* be an error in the simulation that I haven't caught; 100 points for finding one!

In conclusion, ABMs are good tools, but one must be careful in how they're modelled, coded and interpreted.
It's ugly, but it works (Agent-Based Models is the cough syrup of modelling).

There are also more directions available here, with the simple data we gathered:

1) what are the differences in wait times between walk-ins and patients arriving by ambulance? is arrival by ambulance really an advantage here?

2) notice that all the results we gathered so far pertain to the Discharged patients. Let's turn it around: for the Frustrated patients, at what stage did they leave the simulation? Can you use this information to find a bottleneck in the model? Is it what/where you expected?


Happy simulating,

Brendon

b2philli@uwaterloo.ca